# 🚀 Advanced RVC Inference Pro

**State-of-the-art Voice Conversion with KADVC Optimization**

> **Single Source of Truth (SSOT) Colab Notebook**

This notebook provides:
- ⚡ **Dependency Caching** - Skip installation on restarts
- 🗄️ **Drive Mounting** - Auto symlink weights folder
- 🌐 **Tunneling** - Robust Gradio/ngrok integration  
- 🎯 **GPU Auto-detection** - Tesla T4/P100/A100 optimization
- 🧠 **Memory Management** - Automatic OOM prevention
- 🎵 **Voice Processing** - High-quality RVC voice conversion
- 🔧 **Model Loading** - Automatic model detection and loading
- ⚡ **uv Integration** - 10x faster package installation

**Repository:** [ArkanDash/Advanced-RVC-Inference](https://github.com/ArkanDash/Advanced-RVC-Inference)

**Author:** BF667


# Colab Parameters

<!-- This cell contains parameters for the notebook. These parameters should not be modified. -->


#@title ## 📦 Install & Setup Dependencies

This cell includes intelligent caching and uv optimization for the fastest installation!


In [ ]:
#@title Install Dependencies with uv (10x Faster)
#@param {"type": "string", "description": "Optional: Custom CUDA version (leave empty for auto-detect)"}
cuda_version = ""  # e.g., "11.8" or "12.1", leave empty for auto

# ============================================
# ADVANCED RVC DEPENDENCY INSTALLATION
# WITH UV OPTIMIZATION FOR 10X FASTER SETUP
# ============================================

import os
import sys
import time
import subprocess
import json
from pathlib import Path

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🚀 Running in Google Colab environment")
except ImportError:
    IN_COLAB = False
    print("⚠️ Not running in Colab - some features may not work")

# Setup paths
BASE_DIR = Path.cwd()
CACHE_FILE = BASE_DIR / ".colab_deps_cache.json"

# Install CUDA toolkit if specified
if cuda_version and IN_COLAB:
    print(f"🖥️ Installing CUDA {cuda_version} toolkit...")
    subprocess.run(["apt", "update"], check=True)
    subprocess.run(["apt", "install", "-y", f"cuda-toolkit-{cuda_version}"], check=True)

# Define package requirements
REQUIREMENTS = [
    "torch>=2.0.0",
    "torchvision",
    "torchaudio",
    "librosa",
    "soundfile",
    "numpy",
    "scipy",
    "gradio>=3.50.0",
    "transformers",
    "huggingface-hub",
    "omegaconf",
    "numba",
    "ffmpeg-python",
    "audioread",
    "soundfile",
    "resampy",
    "parselmouth",
]

# Cache management
def load_cache():
    if CACHE_FILE.exists():
        with open(CACHE_FILE, "r") as f:
            return json.load(f)
    return {}

def save_cache(cache):
    with open(CACHE_FILE, "w") as f:
        json.dump(cache, f, indent=2)

# Check if packages are already installed
def check_package_installed(package_name):
    try:
        result = subprocess.run(
            ["pip", "show", package_name.split(">=")[0].split("<")[0].split("==")[0]],
            capture_output=True,
            text=True,
            timeout=10
        )
        return result.returncode == 0
    except (subprocess.TimeoutExpired, subprocess.SubprocessError):
        return False

# Main installation logic
print("🔍 Checking existing package installation...")
cache = load_cache()
start_time = time.time()

# Install packages that are not already installed
packages_to_install = []
for req in REQUIREMENTS:
    package_name = req.split(">=")[0].split("<")[0].split("==")[0]
    if not check_package_installed(package_name):
        packages_to_install.append(req)
        print(f"  ➕ Will install: {req}")
    else:
        print(f"  ✅ Already installed: {package_name}")

if packages_to_install:
    print(f"\n📦 Installing {len(packages_to_install)} packages with uv (10x faster)...")
    
    # Install uv for blazing fast package management
    print("⚡ Installing uv package manager...")
    subprocess.run(["pip", "install", "uv"], check=True)
    print("✅ uv installed successfully!")
    
    # Install PyTorch first (important for CUDA)
    torch_reqs = [req for req in packages_to_install if "torch" in req.lower()]
    other_reqs = [req for req in packages_to_install if "torch" not in req.lower()]
    
    if torch_reqs:
        print("🔥 Installing PyTorch components with uv (ultra-fast)...")
        for req in torch_reqs:
            subprocess.run(["uv", "pip", "install", req, "--index-url", "https://download.pytorch.org/whl/cu118"], check=True)
    
    if other_reqs:
        print("📚 Installing other dependencies with uv (lightning fast)...")
        subprocess.run(["uv", "pip", "install"] + other_reqs, check=True)
        
    print("✅ All packages installed successfully!")
else:
    print("\n🎉 All required packages are already installed!")

# Verify critical packages
critical_packages = ["torch", "gradio", "librosa", "uv"]
print("\n🔍 Verifying critical packages...")
for pkg in critical_packages:
    if check_package_installed(pkg):
        print(f"  ✅ {pkg}")
    else:
        print(f"  ❌ {pkg} - Installation failed!")

installation_time = time.time() - start_time
print(f"\n⏱️ Total installation time: {installation_time:.2f} seconds")
print("\n🎉 Dependency installation with uv optimization completed!")
